In [1]:
import os
import pandas as pd
import numpy as np
import nibabel as nib
import nrrd

KeyboardInterrupt: 

In [ ]:
clin_dir = os.path.join(os.path.dirname(os.getcwd()),"data", "clinical_df_cleaned.csv")
clin_df= pd.read_csv(clin_dir)


In [ ]:
test2 = pd.read_csv("radiomics_df_veryfinal.csv")

drop_cases = (set(clin_df["TCIA_ID"].unique())).difference(set(test2["TCIA_ID"].unique()))
clin_df = clin_df[clin_df["TCIA_ID"].isin(drop_cases) == False]

clin_df

,Unnamed: 0,TCIA_ID,Interval_BL,age,Sex,Smoking,Alcohol,fhx_can,fhx_livc,Diabetes,...,Pathology_Moderately differentiated,Pathology_NOT STATED,Pathology_No biopsy,Pathology_Poorly differentiated,Pathology_Well differentiated,HCV,HBV,Child Pugh A,multinodular,T_involvment > 50%
0,0,HCC_001,-18,71,0,1,1,0,0,0,...,1.0,0.0,0.0,0.0,0.0,1,0,1,1,0
1,1,HCC_002,84,72,0,1,1,0,0,0,...,0.0,0.0,0.0,0.0,1.0,0,1,1,1,0
2,2,HCC_003,3,53,0,1,1,1,0,0,...,1.0,0.0,0.0,0.0,0.0,1,0,1,1,1
3,3,HCC_004,1,80,0,0,0,1,0,0,...,1.0,0.0,0.0,0.0,0.0,0,0,1,1,0
4,4,HCC_005,435,73,0,1,1,1,0,0,...,1.0,0.0,0.0,0.0,0.0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,100,HCC_101,-6,57,0,1,1,0,0,0,...,0.0,0.0,0.0,0.0,1.0,1,1,1,0,0
95,101,HCC_102,27,86,0,0,0,1,0,0,...,1.0,0.0,0.0,0.0,0.0,1,0,1,0,0
96,102,HCC_103,1,79,0,1,1,1,0,1,...,0.0,0.0,0.0,0.0,1.0,0,0,1,1,0
97,103,HCC_104,9,66,0,1,1,1,0,1,...,0.0,0.0,0.0,0.0,1.0,1,0,1,1,0


In [ ]:
from scipy.stats import ranksums

def is_train_test_difference_significant(x_train, x_test, alpha=0.05):

    x_train = np.asarray(x_train)
    x_test = np.asarray(x_test)
    
    # Flatten arrays if they are multi-dimensional
    if x_train.ndim > 1:
        x_train = x_train.ravel()
    if x_test.ndim > 1:
        x_test = x_test.ravel()
    
    # Perform the Wilcoxon rank-sum test
    stat, p_value = ranksums(x_train, x_test)
    
    # Check if the difference is statistically significant
    is_significant = p_value < alpha

    
    
    return is_significant, p_value

A/B Testing


combine endpoint and radiomic for initial test


In [ ]:
from sklearn.model_selection import train_test_split

def train_test_split_patients(
    dataframe: pd.DataFrame, 
    identifier: str, 
    endpoint: str, 
    test_ratio: float = 0.2,
    random_state: int = 42
):
    # Get unique patient IDs
    unique_patients = dataframe[identifier].unique()
    
    # Split patients into train and test
    train_patients, test_patients = train_test_split(
        unique_patients, 
        test_size=test_ratio,
        random_state=random_state,
        stratify=dataframe[endpoint]  # Optional: maintain class balance
    )
    
    # Create masks
    train_mask = dataframe[identifier].isin(train_patients)
    test_mask = dataframe[identifier].isin(test_patients)
    
    # Split the data
    x_train = dataframe[train_mask].drop(columns=[endpoint, identifier], axis=1)
    x_test = dataframe[test_mask].drop(columns=[endpoint, identifier], axis=1)
    y_train = dataframe[train_mask][endpoint]
    y_test = dataframe[test_mask][endpoint]

 
    return x_train, x_test, y_train, y_test


In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix

def classification_metrics(y_true, y_pred):
    """
    Compute common classification metrics for binary classification.
    
    Returns a dictionary with:
    - Accuracy
    - F1 Score
    - Precision
    - Sensitivity (Recall)
    - Specificity
    - Confusion Matrix
    """
    acc = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    sensitivity = recall_score(y_true, y_pred)
    
    # Confusion matrix: [[TN, FP], [FN, TP]]
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    specificity = tn / (tn + fp)
    
    metrics_dict = {
        'Accuracy': acc,
        'F1 Score': f1,
        'Precision': precision,
        'Sensitivity (Recall)': sensitivity,
        'Specificity': specificity,
        'Confusion Matrix': confusion_matrix(y_true, y_pred)
    }
    
    return metrics_dict


Standardization of values


In [ ]:


from sklearn.base import BaseEstimator, TransformerMixin

class MissingValueColumnFilter(BaseEstimator, TransformerMixin):
    def __init__(self, threshold=0.3):
        self.threshold = threshold
        self.keep_features_ = None

    def fit(self, X, y=None):
        X = pd.DataFrame(X)
        missing_frac = X.isna().mean()
        self.keep_features_ = missing_frac[missing_frac <= self.threshold].index
        return self

    def transform(self, X):
        X = pd.DataFrame(X)
        # Only keep columns selected during fit
        return X[self.keep_features_]


In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

class CleanFeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, missing_thresh=0.3, variance_thresh=1e-6):
        self.missing_thresh = missing_thresh
        self.variance_thresh = variance_thresh
        self.keep_cols = []

    def fit(self, X, y=None):

        # print("from clean feature selector : X")
        # print(X)
        X = pd.DataFrame(X, columns=X.columns if hasattr(X, "columns") else None)

        # Drop by missing %
        keep_missing = X.isna().mean() < self.missing_thresh
        X2 = X.loc[:, keep_missing]

        # Drop by variance
        var = X2.var()
        keep_var = var > self.variance_thresh

        self.keep_cols_ = X2.columns[keep_var].tolist()
        return self

    def transform(self, X):
        X = pd.DataFrame(X, columns=X.columns if hasattr(X, "columns") else None)

        print(X[self.keep_cols_])
        return X[self.keep_cols_]


In [ ]:
from scipy.stats import mannwhitneyu, kruskal

class FeatureFilterer:
    def __init__(self, responder_idx, nonresponder_idx, filter_method_name = "mannwhitney", p_val_threshold=0.05):
        self.p_val_threshold = p_val_threshold
        self.responder_idx = responder_idx
        self.nonresponder_idx = nonresponder_idx
        self.filter_method_name = filter_method_name
        self.kept_cols = []

    def fit(self, X, y=None):

        # X is already a DataFrame here if CleanFeatureSelector keeps it that way
        X_df = pd.DataFrame(X, index=self.responder_idx.union(self.nonresponder_idx))

        x_responder = X_df.loc[self.responder_idx]
        x_nonresponder = X_df.loc[self.nonresponder_idx]

        self.significant_features = []

        statistical_test = None

        if self.filter_method_name == "mannwhitney":
            statistical_test = mannwhitneyu

        elif self.filter_method_name == "kruskal":
            statistical_test = kruskal
        
        elif self.filter_method_name == "wilcoxon":
            statistical_test = kruskal

        for col in X_df.columns:
            try:
                _, p = statistical_test(
                    x_responder[col].dropna(),
                    x_nonresponder[col].dropna()
                )
                if p < self.p_val_threshold:
                    self.significant_features.append(col)
            except Exception as e:
                print(e)
                continue


        print(self.significant_features)
        return self

    def transform(self, X):
        X_df = pd.DataFrame(X)
        return X_df[self.significant_features]


In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd

class DataFrameWrapper(BaseEstimator, TransformerMixin):
    def __init__(self, columns=None):
        self.columns = columns

    def fit(self, X, y=None):
        # record input column names
        self.columns_ = X.columns if hasattr(X, "columns") else self.columns
        return self

    def transform(self, X):
        return pd.DataFrame(X, columns=self.columns_, index=getattr(X, "index", None))



In [ ]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.impute import SimpleImputer
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_classif

# combined = pd.merge(
#     left=radiomics_df,
#     right= clin_df.drop(columns=['TNM']),
#     on="Subject ID",
#     how="inner"
# )




clin_df = clin_df.drop(columns=["Unnamed: 0"])

x_tr, x_ts, y_tr, y_ts = train_test_split_patients(
    clin_df, 
    identifier="TCIA_ID",
    endpoint="responder", 
    test_ratio=0.4
)
x_tr_index = x_tr.index
x_ts_index = x_ts.index
x_cols = x_tr.columns
    
responder_indices = y_tr[y_tr == 1].index
nonresponder_indices = y_tr[y_tr == 0].index

preprocess_pipe = Pipeline(steps=[
    ("screening", CleanFeatureSelector()),
    ("impute", SimpleImputer(strategy="mean")),
    ("scale", StandardScaler()),
])


# Transform
x_tr_transformed = preprocess_pipe.fit_transform(x_tr)
x_ts_transformed = preprocess_pipe.transform(x_ts)

# Extract kept columns
# screening_cols = preprocess_pipe.named_steps['filter'].keep_cols
kept_cols = preprocess_pipe.named_steps['screening'].keep_cols_

# Final aligned DataFrames
x_tr = pd.DataFrame(x_tr_transformed, index=x_tr.index, columns=kept_cols)
x_ts = pd.DataFrame(x_ts_transformed, index=x_ts.index, columns=kept_cols)

# Now you can use x_tr_final and x_ts_final with their original indices

    Interval_BL  age  Sex  Smoking  Alcohol  fhx_can  fhx_livc  Diabetes  \
0           -18   71    0        1        1        0         0         0   
2             3   53    0        1        1        1         0         0   
4           435   73    0        1        1        1         0         0   
6          -148   71    0        1        1        1         0         0   
7             0   68    0        0        1        1         0         0   
8            -8   59    0        0        0        1         0         0   
9            -1   46    1        1        1        0         0         0   
12            1   61    0        0        0        1         0         1   
13           -7   68    1        1        1        0         0         0   
15            1   49    0        0        1        0         0         1   
16           -2   79    1        0        0        0         0         1   
17            2   50    0        1        1        1         0         1   
19          

In [ ]:
# feature_filter_pipe = Pipeline(steps=[
#     ("filter", FeatureFilterer(
#         p_val_threshold=0.2,
#         responder_idx=responder_indices,
#         nonresponder_idx=nonresponder_indices
#     )),
    
# ])

# # Fit on TRAINING DATA ONLY (after screening + impute)
# x_tr_filtered = feature_filter_pipe.fit_transform(x_tr)
# x_ts_filtered = feature_filter_pipe.transform(x_ts)

# # Extract feature names
# kept_cols = feature_filter_pipe.named_steps["filter"].significant_features

# # Rebuild DataFrame
# x_tr = pd.DataFrame(x_tr_filtered, index=x_tr.index, columns=kept_cols)
# x_ts = pd.DataFrame(x_ts_filtered, index=x_ts.index, columns=kept_cols)


Univariate Analysis



In [ ]:
from mrmr import mrmr_classif
from sklearn.base import BaseEstimator, TransformerMixin

class MRMRSelector(BaseEstimator, TransformerMixin):
    def __init__(self, K=50, show_progress=False):
        self.K = K
        self.show_progress = show_progress
        self.selected_features_ = None

    def fit(self, X, y):
        # mrmr_classif automatically uses mutual information and redundancy internally
        self.selected_features_ = mrmr_classif(
            X, y,
            K=self.K,
            show_progress=self.show_progress
        )
        return self

    def transform(self, X):
        # Ensure that the data type supports column selection
        if isinstance(X, pd.DataFrame):
            return X[self.selected_features_]
        else:
            raise TypeError("MRMRSelector expects a pandas DataFrame as input.")


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier


# logreg = LogisticRegression(
#     max_iter= 5000, 
#     # C=10.0,
#     class_weight= "balanced"
# )

lr_param_grid = {
  "C" :[0.01, 0.1, 1.0, 2.0, 5.0, 10.0, 20.0, 50.0],
  "penalty" : ["l1", "l2"],
  "solver" : ["liblinear", "saga"],
  "class_weight" : [
    "balanced",
    {0:1, 1:2},
    {0:1, 1:5},
    {0:1, 1:10}
  
  ]
}

# XGBoost
xgb_params = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.3],
    'max_depth': [3, 5, 7],
    'min_child_weight': [1, 3, 5],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],

    'scale_pos_weight': [1, (y_tr == 0).sum() / (y_tr == 1).sum()]  # handles class imbalance
}

linear_svc_params = {
    'C': [0.0001, 0.001, 0.01, 0.1, 0.5, 1.0, 5, 10, 20, 100],
    'penalty': ['l1'],
    'loss': ['squared_hinge'],
    'dual': [False],
    'class_weight': [None, 'balanced', {0:1, 1: 2}, {0:1, 1: 5}, {0:1, 1: 10}, {0:1, 1: 20}],
    'max_iter': [5000, 10000, 20000],
    'random_state': [42],
    'intercept_scaling': [0.5, 1.0, 2.0, 3.0]
}
# Support Vector Machine
svm_params = {
    'C': [0.1, 1, 10, 100],
    'gamma': ['scale', 'auto', 0.1, 1, 10],
    'kernel': ['rbf', 'poly', 'sigmoid'],
    'class_weight': ['balanced', {0:1, 1: 5}, {0: 1, 1: 10}]
}

# Extra Trees
et_params = {
    'n_estimators': [100, 200, 500],
    'max_depth': [None, 5, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2'],
    'class_weight': ['balanced',  {0:1, 1:10},   {0:1, 1:20}]
}

rf_params = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'class_weight': ['balanced', 'balanced_subsample',  {0:1, 1:10},   {0:1, 1:20}]
}

mlp_params = {
    'hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50), (64,), (128,), (64, 64), (128, 64)],
    'activation': ['relu', 'tanh'],
    'alpha': np.logspace(2, 4, 10),
    'learning_rate_init': [0.001, 0.01, 0.1],
    'max_iter': [200, 500, 1000],
    'early_stopping': [True],
    'random_state': [42]
}


adaboost_params = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.5, 1.0],
    'estimator': [
        DecisionTreeClassifier(max_depth=1), # Decision Stump
        DecisionTreeClassifier(max_depth=2), 
        DecisionTreeClassifier(max_depth=3)
    ],
    # AdaBoost does not have a native 'scale_pos_weight' parameter. 
    # Imbalance is typically handled by adjusting the class_weight of the base estimator 
    # or relying on the boosting mechanism itself.
    # The DecisionTreeClassifier base estimator must handle the class_weight.
    # Note: GridSearchCV handles the combination.
}

lgbm_params = {
    'n_estimators': [100, 300, 500],
    'learning_rate': [0.01, 0.05, 0.1],
    'num_leaves': [20, 31, 40], # Main complexity parameter for leaf-wise growth
    'max_depth': [-1, 5, 8], # -1 means no limit
    'min_child_samples': [20, 50, 100],
    'subsample': [0.7, 0.9, 1.0], # Row subsampling
    'colsample_bytree': [0.7, 0.9, 1.0], # Feature subsampling
    'reg_alpha': [0, 0.1, 0.5], # L1 regularization
    'reg_lambda': [0, 0.1, 0.5], # L2 regularization
    # Handling Imbalance
    'scale_pos_weight': [1, (y_tr == 0).sum() / (y_tr == 1).sum(), 5, 10, 20]
}

# voting_grid = {
#     "estimators": [
#         ("lr", LogisticRegression(max_iter=5000, class_weight="balanced")),
#         ("xgb", XGBClassifier(use_label_encoder=False, eval_metric="logloss")),
#         ("svc", SVC(probability=True)),
#         ("et", ExtraTreesClassifier()),
#         ("rf", RandomForestClassifier()),
#         ("mlp", MLPClassifier())
#     ]
# }



In [ ]:
def sampling_wrapper(
    x_train,
    y_train,
    sampling_method=None,
) -> tuple[pd.DataFrame, pd.Series]:
    if sampling_method is None:
        return x_train, y_train

    try:
        # Get column names before sampling
        feature_columns = x_train.columns
        target_name = y_train.name if hasattr(y_train, 'name') else 'target'
        
        # Apply sampling
        sampler = sampling_method
        x_tr_res, y_tr_res = sampler.fit_resample(x_train, y_train)
        
        # Convert back to DataFrame/Series with original column names
        x_tr_res = pd.DataFrame(x_tr_res, columns=feature_columns)
        y_tr_res = pd.Series(y_tr_res, name=target_name)
        
        return x_tr_res, y_tr_res

    except Exception as e:
        print(f"Error during sampling: {str(e)}")
        return x_train, y_train

In [ ]:
from sklearn.feature_selection import RFECV

# from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
import pandas as pd
from sklearn.decomposition import SparsePCA
from sklearn.base import clone
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import make_scorer, average_precision_score
from sklearn.model_selection import GridSearchCV, cross_val_predict, StratifiedKFold, RepeatedStratifiedKFold
import numpy as np
import pandas as pd
from tqdm import tqdm
    

def sparse_pca_wrapper(x_train, y_train, estimator, param_grid=None,
                      n_components=None, min_feats=5, alpha=1, ridge_alpha=0.01, 
                      max_iter=1000, n_runs=5, cv=5, scoring='accuracy', 
                      random_state=42, n_jobs=-1):
    """
    Sparse PCA wrapper with integrated cross-validation for feature selection.
    
    Parameters:
    -----------
    x_train : array-like of shape (n_samples, n_features)
        Training data
    y_train : array-like of shape (n_samples,)
        Target values
    estimator : estimator object
        A scikit-learn estimator that implements 'fit' and 'predict'
    param_grid : dict, optional
        Dictionary with parameters names as keys and lists of parameter settings to try
    n_components : int, default=None
        Number of sparse components to extract
    alpha : float, default=1
        Sparsity controlling parameter
    ridge_alpha : float, default=0.01
        Amount of ridge shrinkage
    max_iter : int, default=1000
        Maximum number of iterations
    n_runs : int, default=5
        Number of runs for stability analysis
    cv : int, cross-validation generator, default=5
        Determines cross-validation splitting strategy
    scoring : str, callable, default='accuracy'
        Scoring metric
    random_state : int, default=42
        Random state for reproducibility
    n_jobs : int, default=-1
        Number of jobs to run in parallel
        
    Returns:
    --------
    dict:
        Dictionary containing:
        - 'best_estimator': Best model from GridSearchCV
        - 'best_params': Best parameters from GridSearchCV
        - 'cv_results': Cross-validation results
        - 'feature_importances': Feature importances if available
        - 'stability_scores': Stability of feature selection across runs
    """
  
    if param_grid is None:
        param_grid = {}  # Default empty param grid
    
    # Store results from all runs
    all_importances = []
    best_estimators = []
    
    # Run multiple times for stability analysis
    for run in tqdm(range(n_runs), desc="Running stability analysis"):
        # Set random state for this run
        current_seed = random_state + run if random_state is not None else None
        
        # Set up cross-validation
        cv_splitter = StratifiedKFold(n_splits=cv, shuffle=True, random_state=current_seed)
        
        # Set up GridSearchCV
        grid_search = GridSearchCV(
            estimator=estimator,
            param_grid=param_grid,
            cv=cv_splitter,
            scoring=scoring,
            n_jobs=n_jobs,
            return_train_score=True
        )
        
        # Fit the model
        grid_search.fit(x_train, y_train)
        
        # Store best estimator
        best_estimators.append(grid_search.best_estimator_)
        
        # Get feature importances if available
        if hasattr(grid_search.best_estimator_, 'feature_importances_'):
            importances = grid_search.best_estimator_.feature_importances_
            all_importances.append(importances)
        elif hasattr(grid_search.best_estimator_, 'coef_'):
            # For linear models
            importances = np.abs(grid_search.best_estimator_.coef_).mean(axis=0)
            all_importances.append(importances)
    
    # Calculate stability scores if we have multiple runs
    stability_scores = None
    if n_runs > 1 and all_importances:
        # Convert to numpy array for calculations
        all_importances = np.array(all_importances)
        
        # Calculate stability as coefficient of variation (lower is more stable)
        stability_scores = np.std(all_importances, axis=0) / (np.mean(all_importances, axis=0) + 1e-10)
    
    # Get the best model from the last run
    best_estimator = best_estimators[-1]
    # Get feature importances from the best model
    feature_importances = None
    if hasattr(best_estimator, 'feature_importances_'):
        feature_importances = pd.Series(
            best_estimator.feature_importances_,
            index=x_train.columns if hasattr(x_train, 'columns') else range(x_train.shape[1])
        ).sort_values(ascending=False)
    elif hasattr(best_estimator, 'coef_'):
        # For linear models
        coef = best_estimator.coef_
        if len(coef.shape) > 1:  # For multi-class
            coef = np.abs(coef).mean(axis=0)
        feature_importances = pd.Series(
            coef,
            index=x_train.columns if hasattr(x_train, 'columns') else range(x_train.shape[1])
        ).sort_values(ascending=False)
    

    print(feature_importances)
    return {
        'best_estimator': best_estimator,
        'best_params': grid_search.best_params_,
        'cv_results': grid_search.cv_results_,
        'feature_importances': feature_importances,
        'selected_features' : feature_importances.nlargest(min_feats).index,
        'stability_scores': stability_scores,
        'all_importances': all_importances
    }

def pca_wrapper():
    pca = PCA(n_components=k)
    pca.fit(X)

    # The loadings are here!
    loadings = pca.components_


def lasso_wrapper(x_train, y_train):
    # Define the parameter grid for logistic regression
    param_grid = {
        'Cs': np.logspace(-6, 1, 10),  # Regularization strength,  # Number of regularization values to try
        'penalty': 'l1',
        'solver': 'saga',
        'cv': 5,
        'scoring': 'average_precision',
        'random_state': 42,
        'class_weight': 'balanced',
        'max_iter': 10000,
        'n_jobs': -1,
        'verbose': 1
    }
    
    # Create and fit LogisticRegressionCV
    lr_cv = LogisticRegressionCV(**param_grid)
    lr_cv.fit(x_train, y_train)
    
    # Get best parameters and score
    print(f"Best C: {lr_cv.C_[0]:.4f}")
    print(f"Best penalty: {lr_cv.penalty}")
    print(f"Best cross-validated score: {lr_cv.scores_[1].mean(axis=0).max():.4f}")
    
    # Get feature importance
    feature_importance = pd.DataFrame({
        'feature': x_train.columns,
        'importance': np.abs(lr_cv.coef_[0])
    }).sort_values('importance', ascending=False)
    
    print("\nTop 10 most important features:")
    print(feature_importance.head(10))
    
    return {
        "best_model": lr_cv,
        "feature_importance": feature_importance,
        "selected_features": x_train.columns[lr_cv.coef_[0] != 0].tolist()
    }



def sfs_wrapper(estimator, x_train, y_train, max_feats=None, use_predict_proba=True, min_feats=5):
    """
    Performs Sequential Forward Selection (SFS) using RepeatedStratifiedKFold CV
    to select the most stable and performant features.

    Args:
        estimator: The unfitted model (e.g., LogisticRegression).
        x_train (pd.DataFrame): Training features.
        y_train (pd.Series): Training labels.
        max_feats (int): Maximum number of features to select.
        min_feats (int): Minimum number of features to select (optional stopping point).

    Returns:
        list: The list of selected feature names.
    """
    
    # --- 1. Define the Robust Cross-Validation Strategy ---
    # Using 10 folds repeated 3 times is a common, stable choice.
    # Setting random_state is CRUCIAL for reproducibility of the folds.
    rskf = RepeatedStratifiedKFold(
        n_splits=10, 
        n_repeats=3, 
        random_state=42 
    )

    # --- 2. Initialize the Sequential Feature Selector ---
    # We use Forward Selection ('forward') to build the feature set.
    # We use 'average_precision' as the scoring metric for imbalanced data.
    ap_scorer = make_scorer(
        average_precision_score, 
        needs_proba=True, # 🔑 This is the key argument for using predict_proba
        greater_is_better=True
    )

    # --- 3. Initialize the Sequential Feature Selector ---
    sfs = SequentialFeatureSelector(
        estimator=estimator,
        n_features_to_select="auto" if max_feats == None else max_feats,
        direction='forward',
        # 🔑 Pass the custom scorer object here:
        scoring=ap_scorer, 
        cv=rskf,
        n_jobs=-1,
        # REMOVE the unsupported 'response_method' argument
    )

    # --- 3. Fit the Selector to the Data ---
    # SFS internally performs the cross-validation using rskf at every step
    # to evaluate which feature provides the best, most stable performance gain.
    sfs.fit(x_train, y_train)

    # --- 4. Extract and Return Results ---
    selected_features_mask = sfs.get_support()
    selected_feature_names = list(x_train.columns[selected_features_mask])
  

    return {
        "selected_features" : selected_feature_names
    }    

def rfecv_wrapper(estimator, x_train, y_train, max_feats = 50, min_feats = 10):
    # Ensure x_train is a DataFrame to access column names
    # if not hasattr(x_train, 'columns'):
    #     x_train = pd.DataFrame(x_train)
    
    rfecv = RFECV(
        estimator=estimator,
        step=5,
        cv=StratifiedKFold(3),
        scoring='average_precision',
        n_jobs=-1, 
        min_features_to_select=min_feats,
    )

    rfecv.fit(x_train, y_train)
    
    # Get the selected features by name


    
    # Get feature rankings with names
# First, ensure the features are sorted by importance (best rank first)
    feature_ranking = pd.DataFrame({
        'feature': x_train.columns,
        'ranking': rfecv.ranking_,
        'support': rfecv.support_
    }).sort_values('ranking')  # Sort by ranking (lower rank = more important)

    # Keep only the top max_feats features
    feature_ranking['support'] = feature_ranking.index < max_feats

    # Update selected_features to only include the top max_feats features
    selected_features = feature_ranking[feature_ranking['support']]['feature'].tolist()

    # Print some information
    print(f"Selected top {len(selected_features)} features out of {len(feature_ranking)}")
    print("Selected features:", selected_features)
            
    # Get cross-validation scores for each number of features
    cv_scores = pd.DataFrame({
        'n_features': range(1, len(rfecv.cv_results_['mean_test_score']) + 1),
        'mean_score': rfecv.cv_results_['mean_test_score'],
        'std_score': rfecv.cv_results_['std_test_score']
    })
    
    # selected_features = [x_train.columns[col] for col in selected_features]

    print("Selected features:", selected_features)

    return {
        'selected_features': list(selected_features),
        'feature_ranking': feature_ranking,
        'cv_scores': cv_scores,
        'optimal_n_features': rfecv.n_features_,
        'rfecv': rfecv
    }




In [ ]:

import sys
pipeline_path = os.path.abspath(os.path.join(os.getcwd(), '..'))
if pipeline_path not in sys.path:
    sys.path.append(pipeline_path)

In [ ]:
def plot_cv_metrics(gs_cv_results, n_splits=4, title='Cross-Validation Metrics'):
  
    # Extract metrics for each fold
    metrics = []
    for i in range(1, n_splits):
        fold_metrics = {
            'Fold': i+1,
            'Sensitivity': gs_cv_results[f'split{i}_test_sensitivity'][gs_cv_results['rank_test_accuracy'].argmin()],
            'Specificity': gs_cv_results[f'split{i}_test_specificity'][gs_cv_results['rank_test_accuracy'].argmin()],
            'Accuracy': gs_cv_results[f'split{i}_test_accuracy'][gs_cv_results['rank_test_accuracy'].argmin()]
        }
        metrics.append(fold_metrics)
    
    # Convert to DataFrame for easier plotting
    df_metrics = pd.DataFrame(metrics).melt('Fold', var_name='Metric', value_name='Score')
    
    # Create the plot
    plt.figure(figsize=(12, 6))
    sns.barplot(x='Fold', y='Score', hue='Metric', data=df_metrics)
    plt.title(title)
    plt.ylim(0, 1.1)
    plt.legend(loc='lower right')
    plt.grid(True, alpha=0.3)
    plt.show()
    
    # Print mean and std of metrics
    print(f"Mean Sensitivity: {df_metrics[df_metrics['Metric'] == 'Sensitivity']['Score'].mean():.3f} "
          f"(±{df_metrics[df_metrics['Metric'] == 'Sensitivity']['Score'].std():.3f})")
    print(f"Mean Specificity: {df_metrics[df_metrics['Metric'] == 'Specificity']['Score'].mean():.3f} "
          f"(±{df_metrics[df_metrics['Metric'] == 'Specificity']['Score'].std():.3f})")
    print(f"Mean Accuracy: {df_metrics[df_metrics['Metric'] == 'Accuracy']['Score'].mean():.3f} "
          f"(±{df_metrics[df_metrics['Metric'] == 'Accuracy']['Score'].std():.3f})")

# Example usage:
# plot_cv_metrics(gs.cv_results_, n_splits=4, title='Model Performance per Fold')



In [ ]:
from statistics import LinearRegression
# from custom_models.svm_shap import SVMSHAP
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, RepeatedStratifiedKFold
from sklearn.metrics import classification_report, average_precision_score, accuracy_score, roc_auc_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier #


from xgboost import XGBClassifier
from sklearn.svm import SVC , LinearSVC
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.model_selection import GridSearchCV

from imblearn.over_sampling import ADASYN, SMOTE
from sklearn.model_selection import RepeatedStratifiedKFold, cross_validate

from sklearn.neural_network import MLPClassifier

cv = RepeatedStratifiedKFold(
    n_splits=3,
    n_repeats=5,
    random_state=42
)

# scores = cross_validate(
#     model,
#     X,
#     y,
#     cv=cv,
#     scoring=["roc_auc", "accuracy", "precision", "recall"],
#     n_jobs=-1,
#     return_estimator=False,
#     return_train_score=False
# )

# print(scores["test_roc_auc"].mean(), scores["test_roc_auc"].std())

# XGBoost
xgb_grid = GridSearchCV(
    XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss'),
    xgb_params,
    cv=cv,
    scoring='average_precision',
    n_jobs=-1,
    verbose=1
)

# SVM
svm_grid = GridSearchCV(
    SVC(probability=True, random_state=42),
    svm_params,
    cv=cv,
    scoring='average_precision',
    n_jobs=-1,
    verbose=1
)

linear_svc_grid = GridSearchCV(
    LinearSVC(random_state=42),
    linear_svc_params,
    cv=cv,
    scoring='average_precision',
    n_jobs=-1,
    verbose=1
)

# Extra Trees
et_grid = GridSearchCV(
    ExtraTreesClassifier(random_state=42),
    et_params,
    cv=cv,
    scoring={
        'roc_auc': 'roc_auc',
        'average_precision': 'average_precision',
        'accuracy': 'accuracy',
        # 'f1': 'f1_weighted'  # ← Most common choice
    },
    refit='average_precision',
    n_jobs=-1,
    verbose=1
)

rf_grid = GridSearchCV(
    RandomForestClassifier(random_state=42, n_jobs=-1),
    rf_params,
    cv=cv,
    refit='average_precision',
    scoring={
        'roc_auc': 'roc_auc',
        'average_precision': 'average_precision',
        'accuracy': 'accuracy',
        # 'f1': 'f1_weighted'  # ← Most common choice
    },
    n_jobs=-1,  # Use all available cores
    verbose=1
)
gs = GridSearchCV(
  param_grid=lr_param_grid,
  cv = cv,
  estimator=LogisticRegression(random_state=42),
  refit="accuracy",
  scoring={
    'roc_auc': 'roc_auc',
    'average_precision': 'average_precision',
    'accuracy': 'accuracy'
  },
)


mlp_grid = GridSearchCV(
    MLPClassifier(),
    mlp_params,
    cv=cv,
    scoring={
        'roc_auc': 'roc_auc',
        'average_precision': 'average_precision',
        'accuracy': 'accuracy',
    },
    refit='average_precision',
    n_jobs=-1,  # Note: MLP doesn't support n_jobs > 1
    verbose=1
)

adaboost_grid = GridSearchCV(
    AdaBoostClassifier(estimator=DecisionTreeClassifier(random_state=42), random_state=42),
    adaboost_params,
    cv=cv,
    scoring='average_precision',
    n_jobs=-1,
    verbose=1
)

grids = [
    xgb_grid,        # XGBoost GridSearchCV
    svm_grid,        # SVM GridSearchCV
    linear_svc_grid, # Linear SVC GridSearchCV
    et_grid,         # Extra Trees GridSearchCV
    rf_grid, 
    mlp_grid,        # Random Forest GridSearchCV
    gs               # Logistic Regression GridSearchCV
]


#======= FS WRAPPER ========


    
# current_gs = linear_svc_grid
# current_gs.fit(x_tr, y_tr)
# # Get the best estimator from GridSearchCV
# best_lr = current_gs.best_estimator_

# # Make predictions on the validation set
# y_pred = best_lr.predict(x_ts[feats])
# # Generate and print the classification report
# print("Best Parameters:", current_gs.best_params_)
# print("\nClassification Report for Best Model:")
# print(classification_report(y_ts, y_pred))
# print(confusion_matrix(y_ts,y_pred))
# print("model accuracy:", accuracy_score(y_ts, y_pred))

# x_ts_filtered = x_ts[feats]

# try:
#     # Pass the filtered TEST FEATURES (x_ts[feats]) to predict_proba
#     y_prob = best_lr.predict_proba(x_ts_filtered)
    
#     # NOTE: roc_auc_score requires probabilities for the positive class (column 1)
#     # The output of predict_proba is usually (N_samples, 2), so we take all rows and column index 1
#     model_proba = y_prob[:, 1]
    
#     print("model roc auc", roc_auc_score(y_ts.to_numpy(), model_proba))

# except Exception as e:
#     print(e)
#     print("cannot print proba")

# print(current_gs.cv_results_ )
# # plot_cv_metrics(current_gs.cv_results_, n_splits=3, title='Your Model Performance')



In [ ]:
from sklearn.metrics import make_scorer, roc_auc_score, average_precision_score, accuracy_score, classification_report
from datetime import datetime
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV

def save_model_results(grid, x_tr, y_tr, x_ts, y_ts, feats, 
                      feature_selection_name='', filter_algorithm_name='',
                      output_file='model_results_clinical.xlsx'):
    """
    Save model results including CV metrics (from GridSearchCV) and test set performance.
    Expects a GridSearchCV instance. Will fit the grid if not already fitted.
    """
    try:
        # Ensure grid is fitted (force surfacing of real errors)
        if not hasattr(grid, 'best_estimator_'):
            # If possible, flip error_score to raise to see root cause early
            try:
                grid.error_score = 'raise'
            except Exception:
                pass
            print("Fitting GridSearchCV...")
            grid.fit(x_tr[feats], y_tr)

        best_estimator = grid.best_estimator_
        cv_results = grid.cv_results_
        best_params = grid.best_params_
        best_idx = getattr(grid, 'best_index_', None)
        model_name = type(best_estimator).__name__

        print(f"Best parameters: {best_params}")

        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        run_id = f"{model_name}_{feature_selection_name}_{filter_algorithm_name}_{timestamp}"

        # Extract CV metrics robustly
        # Primary CV score comes from grid.best_score_ regardless of scoring name
        primary_cv = getattr(grid, 'best_score_', None)

        def _get_cv_stat(key_mean, key_std=None):
            if key_mean in cv_results:
                mean_val = cv_results[key_mean][best_idx] if best_idx is not None else np.nan
                if key_std and key_std in cv_results:
                    std_val = cv_results[key_std][best_idx]
                    return f"{mean_val:.4f} ± {std_val:.4f}"
                return f"{mean_val:.4f}"
            return 'N/A'

        # Try to read explicit metric keys if you used a scoring dict when creating the grid
        cv_metrics = {
            'cv_primary_metric': f"{primary_cv:.4f}" if primary_cv is not None else 'N/A',
            'cv_train_accuracy': _get_cv_stat('mean_train_accuracy', 'std_train_accuracy'),
            'cv_test_accuracy':  _get_cv_stat('mean_test_accuracy',  'std_test_accuracy'),
            'cv_train_roc_auc':  _get_cv_stat('mean_train_roc_auc',  'std_train_roc_auc'),
            'cv_test_roc_auc':   _get_cv_stat('mean_test_roc_auc',   'std_test_roc_auc'),
            'cv_train_ap':       _get_cv_stat('mean_train_average_precision', 'std_train_average_precision'),
            'cv_test_ap':        _get_cv_stat('mean_test_average_precision',  'std_test_average_precision'),
            # Fallback if you passed a single scoring like 'average_precision'
            'cv_test_score':     _get_cv_stat('mean_test_score', 'std_test_score')
        }

        # Test set predictions
        y_pred = best_estimator.predict(x_ts[feats])
        y_prob = best_estimator.predict_proba(x_ts[feats])[:, 1] if hasattr(best_estimator, 'predict_proba') else None

        test_metrics = {
            'test_accuracy': accuracy_score(y_ts, y_pred),
            'test_roc_auc': roc_auc_score(y_ts, y_prob) if y_prob is not None else None,
            'test_average_precision': average_precision_score(y_ts, y_prob) if y_prob is not None else None,
        }

        clf_report = classification_report(y_ts, y_pred, output_dict=True)

        metrics_data = {
            'Run_ID': run_id,
            'Model': model_name,
            'Feature_Selection': feature_selection_name,
            'Filter_Algorithm': filter_algorithm_name,
            'Num_Features': len(feats),
            'Timestamp': timestamp,
            'CV_Strategy': 'GridSearchCV',
            **cv_metrics,
            **{k: v for k, v in test_metrics.items()},
            'Features_Used': str(feats),
            'Model_Params': str(best_params)
        }
        metrics_df = pd.DataFrame([metrics_data])
        clf_df = pd.DataFrame(clf_report).transpose()
        clf_df['Run_ID'] = run_id
        clf_df = clf_df.reset_index().rename(columns={'index': 'class'})

        # Save to Excel
        try:
            with pd.ExcelWriter(output_file, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
                try:
                    existing_metrics = pd.read_excel(writer, sheet_name='Metrics')
                    metrics_df = pd.concat([existing_metrics, metrics_df], ignore_index=True)
                except Exception:
                    pass
                metrics_df.to_excel(writer, sheet_name='Metrics', index=False)

                try:
                    existing_clf = pd.read_excel(writer, sheet_name='Classification_Reports')
                    clf_df = pd.concat([existing_clf, clf_df], ignore_index=True)
                except Exception:
                    pass
                clf_df.to_excel(writer, sheet_name='Classification_Reports', index=False)
        except Exception:
            with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
                metrics_df.to_excel(writer, sheet_name='Metrics', index=False)
                clf_df.to_excel(writer, sheet_name='Classification_Reports', index=False)

        print(f"Results saved to {output_file}")
        return metrics_df, clf_df

    except Exception as e:
        print(f"Error saving results: {str(e)}")
        import traceback
        traceback.print_exc()
        return None, None

In [ ]:
print(x_tr.shape)

(57, 28)


In [ ]:
# First, run the lasso_wrapper to get the selected features
from custom_models.svm_shap import SVMSHAP
from sklearn.feature_selection import SequentialFeatureSelector
# lasso_result = lasso_wrapper(
    
#     x_train=x_tr,
#     y_train=y_tr
# )

# # Get the selected features from Lasso
# selected_features_lasso = lasso_result["selected_features"]

# Use the selected features from Lasso as input to RFECV
rfecv_result = sfs_wrapper(
    estimator=AdaBoostClassifier(random_state=42),  # or your preferred estimator
    x_train=x_tr,  # Use only Lasso-selected features
    y_train=y_tr,
    # max_feats=10,
    use_predict_proba=True
)

# rfecv_result = rfecv_wrapper(
#     estimator=AdaBoostClassifier(random_state=42),
#     x_train = x_tr,
#     y_train = y_tr, 
#     max_feats=10
# )

# Get the final selected features from RFECV
selected_features_rfecv  = rfecv_result["selected_features"]
# print("Lasso selected features:", selected_features_lasso)
print("RFECV selected features:", selected_features_rfecv)
# # Get the selected features
# selected_features = result["selected_features"]
metrics_df, clf_report = save_model_results(
    grid=svm_grid,  # Pass the fitted GridSearchCV instance
    x_tr=x_tr,
    y_tr=y_tr,
    x_ts=x_ts,
    y_ts=y_ts,
    feats=selected_features_rfecv,
    feature_selection_name='None',  # or your feature selection method
    filter_algorithm_name='MannWhitney',  # or your filter method
    output_file='model_results_clinical.xlsx'
)

KeyboardInterrupt: 

: 

In [ ]:
import numpy as np
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score
from sklearn.linear_model import LogisticRegression

# -----------------------------
# Replace these with your data
# X = your feature matrix
# y = your label vector
# -----------------------------

clf = LogisticRegression(max_iter=5000)
cv = RepeatedStratifiedKFold(
    n_splits=5,
    n_repeats=5,
    random_state=0
)

# ---- Real score ----
real_scores = cross_val_score(
    clf,
    x_tr,
    y_tr,
    scoring="roc_auc",
    cv=cv,
    n_jobs=-1
)

print("Real AUC:", real_scores.mean(), "±", real_scores.std())

# ---- Permuted score ----
y_perm = np.random.permutation(y_tr)

perm_scores = cross_val_score(
    clf,
    x_tr,
    y_perm,
    scoring="roc_auc",
    cv=cv,
    n_jobs=-1
)

print("Permuted AUC:", perm_scores.mean(), "±", perm_scores.std())


Real AUC: 0.5978571428571429 ± 0.17000393902999508
Permuted AUC: 0.49660714285714286 ± 0.18332198867187818


Model Tuning(?
)

In [ ]:
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

def calculate_vif(df: pd.DataFrame) -> pd.DataFrame:
    """
    Calculates the Variance Inflation Factor (VIF) for all numeric columns in a DataFrame.
    """
    # Use only numeric, non-null columns
    X = df.select_dtypes(include=np.number).dropna(axis=1, how='any')

    if X.empty:
        return pd.DataFrame({'Feature': [], 'VIF': []})

    # Add constant for intercept term
    X_vif = sm.add_constant(X)
    
    features = X.columns
    vif_list = []

    for i in range(len(features)):
        # Calculate VIF. Index i+1 accounts for the added 'const' column at index 0.
        try:
            vif = variance_inflation_factor(X_vif.values, i + 1)
            vif_list.append(vif)
        except Exception:
            vif_list.append(np.nan) # Set to NaN if calculation fails (e.g., perfect collinearity)

    vif_df = pd.DataFrame({'Feature': features, 'VIF': vif_list})
    return vif_df.sort_values(by='VIF', ascending=False).reset_index(drop=True)

vif_df = calculate_vif(x_tr)

RFE
